In [1]:
import pandas as pd
import datetime
import os

In [2]:
dfm = pd.read_csv('data/dataset15.csv', sep='\t')
record_num = len(dfm)
dfm['date'] = dfm.Time.map(lambda x:datetime.date.fromtimestamp(x))
dfm.head(3)

print(record_num)
dfm.head(3)

45881450


,UserId,ItemId,SessionId,Time,date
0,1,181459,0,1.444342e+09,2015-10-09
1,1,779078,0,1.444342e+09,2015-10-09
2,1,779078,0,1.444342e+09,2015-10-09


In [3]:
U2Freq = {}
for row in dfm.itertuples():
    U2Freq.setdefault(row.UserId, 0)
    U2Freq[row.UserId] += 1
print(len(U2Freq))    

422282


In [9]:
threshold = 59
U1 = []
U2 = []
for UserId, freq in U2Freq.items():
    if freq >= threshold:
        U1.append(UserId)
    else:
        U2.append(UserId)
print(len(U1), len(U2))    

211037 211245


In [10]:
dfm1 = dfm[dfm.UserId.isin(set(U1))]
dfm2 = dfm[dfm.UserId.isin(set(U2))]
print(len(dfm1), len(dfm2))

39839519 6041931


In [11]:
start_date = datetime.date(2015,5,20)
end_date = datetime.date(2015,11,12)

target_start_date = start_date + datetime.timedelta(28)

RF2N1 = {}
RF2H1 = {}
for i in range((end_date - target_start_date).days + 1):
    target_date = target_start_date + datetime.timedelta(i)
    train_start_date = target_date - datetime.timedelta(28) 
    train_end_date = target_date - datetime.timedelta(1) 
    if i>146:break
    print(i, train_start_date.strftime("%F"), '~', train_end_date.strftime("%F"), ':', target_date.strftime("%F"), flush=True)  
    x_dfm = dfm1[(train_start_date <= dfm1.date) & (dfm1.date <= train_end_date)].copy()
    y_dfm = dfm1[dfm1.date == target_date].copy()
    x_record_num = len(x_dfm)
    y_record_num = len(y_dfm)
    print(x_record_num, y_record_num)
    x_dfm['target_date'] = target_date
    x_dfm['day_rcen'] = (x_dfm.target_date - x_dfm.date).map(lambda x:x.days)

    U2Is = {}
    for row in y_dfm.itertuples():
        U2Is.setdefault(row.UserId, set())
        U2Is[row.UserId].add(row.ItemId)

    U2I2Info = {}
    for row in x_dfm.itertuples():
        U2I2Info.setdefault(row.UserId, {})
        U2I2Info[row.UserId].setdefault(row.ItemId, [])
        U2I2Info[row.UserId][row.ItemId].append(row.day_rcen)

    for UserId, I2Info in U2I2Info.items():
        HitSet = set(U2Is[UserId]) if UserId in U2Is else set()
        for ItemId, Info in I2Info.items():
            freq = len(Info)
            rcen = min(Info)
            RF2N1.setdefault((rcen,freq), 0)
            RF2H1.setdefault((rcen,freq), 0)
            RF2N1[rcen,freq] += 1
            if ItemId in HitSet:
                RF2H1[rcen,freq] += 1
    N = 0
    for rf, n in RF2N1.items():
        N += n
    print('data:', N)
RF2P1 = {}
for rf, n in RF2N1.items():
    RF2P1[rf] = RF2H1[rf] / n                    

0 2015-05-20 ~ 2015-06-16 : 2015-06-17
3927143 256682
data: 2655184
1 2015-05-21 ~ 2015-06-17 : 2015-06-18
4020181 199827
data: 5360292
2 2015-05-22 ~ 2015-06-18 : 2015-06-19
4049460 150300
data: 8072101
3 2015-05-23 ~ 2015-06-19 : 2015-06-20
4009920 148259
data: 10760931
4 2015-05-24 ~ 2015-06-20 : 2015-06-21
3982078 142246
data: 13434311
5 2015-05-25 ~ 2015-06-21 : 2015-06-22
3963350 137228
data: 16097251
6 2015-05-26 ~ 2015-06-22 : 2015-06-23
3961963 135169
data: 18763196
7 2015-05-27 ~ 2015-06-23 : 2015-06-24
3963116 168675
data: 21434461
8 2015-05-28 ~ 2015-06-24 : 2015-06-25
3980818 214480
data: 24121618
9 2015-05-29 ~ 2015-06-25 : 2015-06-26
4039461 462351
data: 26850604
10 2015-05-30 ~ 2015-06-26 : 2015-06-27
4350953 261910
data: 29760540
11 2015-05-31 ~ 2015-06-27 : 2015-06-28
4472497 236791
data: 32745888
12 2015-06-01 ~ 2015-06-28 : 2015-06-29
4573858 112551
data: 35793781
13 2015-06-02 ~ 2015-06-29 : 2015-06-30
4560624 100322
data: 38831564
14 2015-06-03 ~ 2015-06-30 : 2015

In [13]:
start_date = datetime.date(2015,5,20)
end_date = datetime.date(2015,11,12)

target_start_date = start_date + datetime.timedelta(28)

RF2N2 = {}
RF2H2 = {}
for i in range((end_date - target_start_date).days + 1):
    target_date = target_start_date + datetime.timedelta(i)
    train_start_date = target_date - datetime.timedelta(28) 
    train_end_date = target_date - datetime.timedelta(1) 
    if i>146:break
    print(i, train_start_date.strftime("%F"), '~', train_end_date.strftime("%F"), ':', target_date.strftime("%F"), flush=True)  
    x_dfm = dfm2[(train_start_date <= dfm2.date) & (dfm2.date <= train_end_date)].copy()
    y_dfm = dfm2[dfm2.date == target_date].copy()
    x_record_num = len(x_dfm)
    y_record_num = len(y_dfm)
    print(x_record_num, y_record_num)
    x_dfm['target_date'] = target_date
    x_dfm['day_rcen'] = (x_dfm.target_date - x_dfm.date).map(lambda x:x.days)

    U2Is = {}
    for row in y_dfm.itertuples():
        U2Is.setdefault(row.UserId, set())
        U2Is[row.UserId].add(row.ItemId)

    U2I2Info = {}
    for row in x_dfm.itertuples():
        U2I2Info.setdefault(row.UserId, {})
        U2I2Info[row.UserId].setdefault(row.ItemId, [])
        U2I2Info[row.UserId][row.ItemId].append(row.day_rcen)

    for UserId, I2Info in U2I2Info.items():
        HitSet = set(U2Is[UserId]) if UserId in U2Is else set()
        for ItemId, Info in I2Info.items():
            freq = len(Info)
            rcen = min(Info)
            RF2N2.setdefault((rcen,freq), 0)
            RF2H2.setdefault((rcen,freq), 0)
            RF2N2[rcen,freq] += 1
            if ItemId in HitSet:
                RF2H2[rcen,freq] += 1
    N = 0
    for rf, n in RF2N2.items():
        N += n
    print('data:', N)
RF2P2 = {}
for rf, n in RF2N2.items():
    RF2P2[rf] = RF2H2[rf] / n                    

0 2015-05-20 ~ 2015-06-16 : 2015-06-17
493496 26307
data: 346035
1 2015-05-21 ~ 2015-06-17 : 2015-06-18
499114 24075
data: 695782
2 2015-05-22 ~ 2015-06-18 : 2015-06-19
501109 17293
data: 1046494
3 2015-05-23 ~ 2015-06-19 : 2015-06-20
494978 16667
data: 1392855
4 2015-05-24 ~ 2015-06-20 : 2015-06-21
491296 15583
data: 1736817
5 2015-05-25 ~ 2015-06-21 : 2015-06-22
487337 15063
data: 2077893
6 2015-05-26 ~ 2015-06-22 : 2015-06-23
483604 15159
data: 2416478
7 2015-05-27 ~ 2015-06-23 : 2015-06-24
479062 16327
data: 2752343
8 2015-05-28 ~ 2015-06-24 : 2015-06-25
475843 17437
data: 3086325
9 2015-05-29 ~ 2015-06-25 : 2015-06-26
473845 27109
data: 3419318
10 2015-05-30 ~ 2015-06-26 : 2015-06-27
482727 21467
data: 3758878
11 2015-05-31 ~ 2015-06-27 : 2015-06-28
486029 18351
data: 4101005
12 2015-06-01 ~ 2015-06-28 : 2015-06-29
488387 13169
data: 4444986
13 2015-06-02 ~ 2015-06-29 : 2015-06-30
485801 12951
data: 4787227
14 2015-06-03 ~ 2015-06-30 : 2015-07-01
482748 14820
data: 5127290
15 2015

In [14]:
g = open('output/fr2p_royal.csv', 'w')
g.write('freq,rcen,prob\n')
for rf, p in sorted(RF2P1.items()):
    r, f = rf
    if f > 40:continue
    print(f, r, p)
    g.write('%d,%d,%f\n' % (f, r, p))
g.close()

1 1 0.018493769198969088
2 1 0.052252263417838396
3 1 0.08204068701859817
4 1 0.10231327812581424
5 1 0.1169452691364399
6 1 0.13051800761564594
7 1 0.14081040287936838
8 1 0.15056677053696427
9 1 0.15997379312548937
10 1 0.16840374230538457
11 1 0.17639597741068463
12 1 0.1831779408847451
13 1 0.19034075946700374
14 1 0.19917321767416643
15 1 0.20733183468787827
16 1 0.21685698749320284
17 1 0.22139137401522233
18 1 0.22283241803960727
19 1 0.23126903553299494
20 1 0.2284440039643211
21 1 0.24467153284671533
22 1 0.23970170454545456
23 1 0.266750418760469
24 1 0.2715605749486653
25 1 0.28215527230590964
26 1 0.2933425797503467
27 1 0.2968868640850418
28 1 0.27912824811399833
29 1 0.2650485436893204
30 1 0.27534562211981567
31 1 0.2998678996036988
32 1 0.3208955223880597
33 1 0.27960526315789475
34 1 0.29924242424242425
35 1 0.3006535947712418
36 1 0.2849740932642487
37 1 0.32642487046632124
38 1 0.2873900293255132
39 1 0.28753993610223644
40 1 0.27586206896551724
1 2 0.010508049516980

In [15]:
g = open('output/fr2p_normal.csv', 'w')
g.write('freq,rcen,prob\n')
for rf, p in sorted(RF2P2.items()):
    r, f = rf
    if f > 40:continue
    print(f, r, p)
    g.write('%d,%d,%f\n' % (f, r, p))
g.close()

1 1 0.009330461095425043
2 1 0.027036806062174946
3 1 0.03951531528450001
4 1 0.044605732105732106
5 1 0.05219471528507739
6 1 0.05084970791290494
7 1 0.05888612983327421
8 1 0.057660626029654036
9 1 0.06452750352609309
10 1 0.06847764572722127
11 1 0.07242582897033159
12 1 0.07152317880794702
13 1 0.08677685950413223
14 1 0.06837606837606838
15 1 0.06349206349206349
16 1 0.07051282051282051
17 1 0.08928571428571429
18 1 0.1134020618556701
19 1 0.07142857142857142
20 1 0.11320754716981132
21 1 0.13953488372093023
22 1 0.09375
23 1 0.0
24 1 0.05
25 1 0.0
26 1 0.2727272727272727
27 1 0.0
28 1 0.0
29 1 0.0
30 1 0.2
31 1 0.0
32 1 0.2
36 1 0.0
38 1 0.0
1 2 0.005050657723546876
2 2 0.01397190534552654
3 2 0.02106345911483431
4 2 0.025074518649802625
5 2 0.0274956356133947
6 2 0.028959210335317172
7 2 0.03214845791949817
8 2 0.03493351363533919
9 2 0.035742035742035744
10 2 0.03400503778337532
11 2 0.0419921875
12 2 0.043923865300146414
13 2 0.037383177570093455
14 2 0.04643962848297214
15 2 